In [1]:
'''
一致性检验程序，无需安装环境。
输入数据的三列依次是葛老师、王老师、周老师。
'''

'\n一致性检验程序，无需安装环境。\n输入数据的三列依次是葛老师、王老师、周老师。\n'

In [2]:
# 百分制，Kendall's W

import numpy as np
from scipy.stats import kendalltau
import scipy.stats as stats

def read_data(file_path):
    data = np.loadtxt(file_path)
    object_ids = data[:, 0]
    scores = data[:, 1:]
    return object_ids, scores

def kendalls_w(scores):
    num_objects, num_raters = scores.shape
    rank_sums = np.sum(scores, axis=1)
    mean_rank = np.mean(rank_sums)
    S = np.sum((rank_sums - mean_rank)**2)
    W = 12 * S / (num_raters**2 * (num_objects**3 - num_objects))
    return W

# 读取数据文件
file_path = 'ConsistencyPercentage.txt'
object_ids, scores = read_data(file_path)

# 将评分转化为排名
ranks = np.argsort(np.argsort(scores, axis=0), axis=0) + 1

# 计算Kendall's W
W = kendalls_w(ranks)
print(f"Kendall's W: {W}")


Kendall's W: 0.4659745396119145


In [3]:
# 百分制，加归一化
# 对每个评分者，取其最低打分为0分，最高打分为100分，将其打分区间均匀映射到[0,100]上
# 归一化对Kendall's W并无影响，因为Kendall's W本质上不是检验分数的一致性而是检验排序的一致性

# 读取数据文件
file_path = 'ConsistencyPercentage.txt'
object_ids, scores = read_data(file_path)


# 初始化一个相同大小的数组来存放归一化后的数据
normalized_data = np.zeros_like(scores, dtype=float)

# 对每一列进行归一化处理
for i in range(scores.shape[1]):
    min_val = np.min(scores[:, i])
    max_val = np.max(scores[:, i])
    
    # 避免除以零的情况
    if max_val != min_val:
        normalized_data[:, i] = (scores[:, i] - min_val) / (max_val - min_val) * 100
    else:
        normalized_data[:, i] = 0  # 如果 max == min，则整列为相同值，可以设为0


# 将评分转化为排名
ranks = np.argsort(np.argsort(scores, axis=0), axis=0) + 1

# 计算Kendall's W
W = kendalls_w(ranks)
print(f"Kendall's W: {W}")

Kendall's W: 0.4659745396119145


In [4]:
# 等级制，Fleiss Kappa
'''
# 没给我这部分数据，我只能写码不能运算
# 如果直接把百分制换算成5级制，结果一塌糊涂，这侧面说明专家打分的心理会受到分数制度的影响，并且对不同专家来说这种影响还不一样
# ————必须先对专家平时工作中打分的心理、情景、分数制度做调查研究，别急着规定用什么分数制度
'''

import numpy as np

def read_data(file_path):
    # 读取txt文件
    data = np.loadtxt(file_path)
    # 提取评分部分
    scores = data[:, 1:]
    return scores

# 定义百分制转五等制的函数，正式版要注释掉，并修改最下方用于计算的代码
def convert_scores(scores):
    # 定义转换函数
    def convert_score(score):
        if 0 <= score <= 20:
            return 1
        elif 21 <= score <= 40:
            return 2
        elif 41 <= score <= 60:
            return 3
        elif 61 <= score <= 80:
            return 4
        elif 81 <= score <= 100:
            return 5
        else:
            raise ValueError("Score out of range")
    # 向量化转换函数
    vectorized_convert = np.vectorize(convert_score)
    # 转换评分
    converted_scores = vectorized_convert(scores)
    return converted_scores

def fleiss_kappa(matrix,k):
    num_objects, num_raters = matrix.shape
    # 计算每个类别在每个对象上的频率
    category_count = np.zeros((num_objects, k))
    for i in range(num_objects):
        for j in range(num_raters):
            category_count[i, int(matrix[i, j]) - 1] += 1
    # 计算p_j（各类别总体比例）
    p_j = np.sum(category_count, axis=0) / (num_objects * num_raters)
    # 计算P_i（各对象的一致性）
    P_i = (np.sum(category_count**2, axis=1) - num_raters) / (num_raters * (num_raters - 1))
    # 计算总体一致性P_bar
    P_bar = np.mean(P_i)
    # 计算期望一致性P_e
    P_e = np.sum(p_j**2)
    # 计算Fleiss Kappa
    kappa = (P_bar - P_e) / (1 - P_e)
    return kappa

# 读取数据文件
file_path = 'ConsistencyPercentage.txt'
scores = read_data(file_path)

# 计算Fleiss Kappa
converted_scores = convert_scores(scores)  # 正式版要注释掉
k = 5 # 5级制
kappa = fleiss_kappa(converted_scores,k)  # 正式版里converted_scores改成scores
print(f"Fleiss Kappa: {kappa}")


Fleiss Kappa: -0.19271120927664218


In [5]:
# 之前结果为负数，这表示各专家完全达不成一致
# 为了验证代码是否有误，这组测试数据是三人完全一致，结果是1——不是代码的问题
TestMatrix = np.zeros([30,3])
for i in range(30):
    TestMatrix[i,:] = int(i%5 +1), int(i%5 +1), int(i%5 +1)
    
kappa = fleiss_kappa(TestMatrix,5)
print(f"Fleiss Kappa: {kappa}")

Fleiss Kappa: 1.0
